In [14]:
import time
def read_soduku(filename):
    puzzles = []
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if len(line) == 81 and line.isdigit():
                board = [[int(line[i*9 + j]) for j in range(9)] for i in range(9)]
                puzzles.append(board)
    return puzzles

def print_board(board):
    for i in range(9):
        for j in range(9):
            print(board[i][j], end=" ")
        print()

def find_empty(board):
    for i in range(9):
        for j in range(9):
            if board[i][j]==0:
                return (i, j)  
    return None

def is_valid(board, num, pos):
    row, col = pos
    for i in range(9):
        if board[i][col]==num and i!=row:
            return False
    for j in range(9):
        if board[row][j]==num and j !=col:
            return False
    # for i in range(9):
    #     if board[i][col] == num and i != row:
    #         return False
    box_x=col//3
    box_y=row//3
    for i in range(box_y * 3, box_y * 3 + 3):
        for j in range(box_x * 3, box_x * 3 + 3):
            if board[i][j] ==num and (i, j)!=pos:
                return False

    return True

def solve(board):
    empty = find_empty(board)
    if not empty:
        return True  
    row, col = empty

    for num in range(1, 10):
        if is_valid(board, num, (row, col)):
            board[row][col] = num

            if solve(board):
                return True

            board[row][col] = 0

    return False

# board = [
#     [0,0,3,0,2,0,6,0,0],
#     [9,0,0,3,0,5,0,0,1],
#     [0,0,1,8,0,6,4,0,0],
#     [0,0,8,1,0,2,9,0,0],
#     [7,0,0,0,0,0,0,0,8],
#     [0,0,6,7,0,8,2,0,0],
#     [0,0,2,6,0,9,5,0,0],
#     [8,0,0,2,0,3,0,0,9],
#     [0,0,5,0,1,0,3,0,0],
# ]

soduku = read_soduku("input.txt")
    
for idx, puzzle in enumerate(soduku):
        print("Original Board:")
        print_board(puzzle)

        start = time.time()
        if solve(puzzle):
            end = time.time()
            print("\n✅ Solved Board:")
            print_board(puzzle)
            print(f"Time: {end - start:.6f} seconds")
        else:
            print("❌ No solution found.")

Original Board:
4 8 3 9 2 1 6 5 7 
9 6 7 3 4 5 8 2 1 
2 5 1 8 7 6 4 9 3 
5 4 8 1 3 2 9 7 6 
7 2 9 5 6 4 1 3 8 
1 3 6 7 9 8 2 4 5 
3 7 2 6 8 9 5 1 4 
8 1 4 2 5 3 7 6 9 
6 9 5 4 1 7 3 8 2 

✅ Solved Board:
4 8 3 9 2 1 6 5 7 
9 6 7 3 4 5 8 2 1 
2 5 1 8 7 6 4 9 3 
5 4 8 1 3 2 9 7 6 
7 2 9 5 6 4 1 3 8 
1 3 6 7 9 8 2 4 5 
3 7 2 6 8 9 5 1 4 
8 1 4 2 5 3 7 6 9 
6 9 5 4 1 7 3 8 2 
Time: 0.000006 seconds


In [6]:
from collections import deque
import copy
import time
# Constants
ROWS = COLS = 9
DIGITS = '123456789'
ALL_CELLS = [(r, c) for r in range(ROWS) for c in range(COLS)]

def parse_puzzle(line):
    puzzle = {}
    for idx, ch in enumerate(line.strip()):
        r, c = divmod(idx, 9)
        if ch in DIGITS:
            puzzle[(r, c)] = ch
        else:
            puzzle[(r, c)] = DIGITS
    return puzzle

def get_peers():
    peers = {}
    for r in range(9):
        for c in range(9):
            peer_set = set()
            for i in range(9):
                peer_set.add((r, i))  # Row
                peer_set.add((i, c))  # Column
            br, bc = 3 * (r // 3), 3 * (c // 3)
            for i in range(3):
                for j in range(3):
                    peer_set.add((br + i, bc + j))  # Box
            peer_set.remove((r, c))
            peers[(r, c)] = peer_set
    return peers

PEERS = get_peers()

def ac3(puzzle):
    queue = deque([(xi, xj) for xi in ALL_CELLS for xj in PEERS[xi]])
    while queue:
        xi, xj = queue.popleft()
        if revise(puzzle, xi, xj):
            if len(puzzle[xi]) == 0:
                return False
            for xk in PEERS[xi] - {xj}:
                queue.append((xk, xi))
    return True

def revise(puzzle, xi, xj):
    revised = False
    if len(puzzle[xj]) == 1:
        if puzzle[xj] in puzzle[xi]:
            puzzle[xi] = puzzle[xi].replace(puzzle[xj], '')
            revised = True
    return revised

def is_complete(puzzle):
    return all(len(puzzle[cell]) == 1 for cell in ALL_CELLS)

def select_unassigned_var(puzzle):
    return min((cell for cell in ALL_CELLS if len(puzzle[cell]) > 1), key=lambda c: len(puzzle[c]))

def backtrack(puzzle):
    if is_complete(puzzle):
        return puzzle
    var = select_unassigned_var(puzzle)
    for value in puzzle[var]:
        new_puzzle = copy.deepcopy(puzzle)
        new_puzzle[var] = value
        if ac3(new_puzzle):
            result = backtrack(new_puzzle)
            if result:
                return result
    return None

def solve(puzzle_line):
    puzzle = parse_puzzle(puzzle_line)
    if not ac3(puzzle):
        return None
    solution = backtrack(puzzle)
    if solution:
        return ''.join(solution[(r, c)] for r in range(9) for c in range(9))
    return None

# === MAIN EXECUTION ===
input_filename = 'input.txt'

try:
    with open(input_filename, 'r') as infile:
        puzzles = infile.readlines()

    for i, puzzle in enumerate(puzzles):
        solution = solve(puzzle)
        if solution:
            print(solution)
            start = time.time()
        if solve(puzzle):
            end = time.time()
            print(f"Time: {end - start:.6f} seconds")
        else:
            print("No solution found for puzzle", i+1)

except FileNotFoundError:
    print(f"File '{input_filename}' not found. Please make sure it's in the same folder as the script.")


483921657967345821251876493548132976729564138136798245372689514814253769695417382
Time: 0.000506 seconds


In [9]:
pip install ortools


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
#google or tools
from ortools.sat.python import cp_model
import time

time_taken = time.time()

def solve_sudoku_or_tools(grid_string):
    model = cp_model.CpModel()
    cell = {}
    for i in range(9):
        for j in range(9):
            cell[(i, j)] = model.NewIntVar(1, 9, f'cell_{i}_{j}')

    for i in range(9):
        model.AddAllDifferent([cell[(i, j)] for j in range(9)])  # Rows
        model.AddAllDifferent([cell[(j, i)] for j in range(9)])  # Columns

    for block_row in range(3):
        for block_col in range(3):
            block = []
            for i in range(3):
                for j in range(3):
                    block.append(cell[(block_row * 3 + i, block_col * 3 + j)])
            model.AddAllDifferent(block)  # Blocks

    for i in range(9):
        for j in range(9):
            char = grid_string[i * 9 + j]
            if char in '123456789':
                model.Add(cell[(i, j)] == int(char))

    solver = cp_model.CpSolver()
    start = time.time()
    status = solver.Solve(model)
    end = time.time()

    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        solution = ''
        for i in range(9):
            for j in range(9):
                solution += str(solver.Value(cell[(i, j)]))
        return solution, end - start
    else:
        return None, end - start
print(f"Time Taken: {time.time() - time_taken}")

Time Taken: 0.00033402442932128906
